In [1]:
tools.selTournament?=

SyntaxError: invalid syntax (<ipython-input-1-4336c76cc05b>, line 1)

In [2]:
import operator
import itertools
import numpy as np

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.datasets import load_digits
from sklearn.cross_validation import train_test_split

np.seterr(all='raise')

digits = load_digits()
digit_features, digit_labels = digits.data, digits.target


X_train_tot, X_test_tot, y_train_tot, y_test_tot = train_test_split(digit_features, digit_labels, stratify=digit_labels,
                                                    train_size=0.75, test_size=0.25)


X_train, X_test, y_train, y_test = train_test_split(X_train_tot, y_train_tot, stratify=y_train_tot,
                                                    train_size=0.75, test_size=0.25)




# defined a new primitive set for strongly typed GP
pset = gp.PrimitiveSetTyped('MAIN', itertools.repeat(float, digit_features.shape[1]), bool, 'Feature')

# boolean operators
pset.addPrimitive(operator.and_, [bool, bool], bool)
pset.addPrimitive(operator.or_, [bool, bool], bool)
pset.addPrimitive(operator.not_, [bool], bool)

# floating point operators
# Define a protected division function
def protectedDiv(left, right):
    try: return left / right
    except (ZeroDivisionError, FloatingPointError): return 1.

pset.addPrimitive(operator.add, [float, float], float)
pset.addPrimitive(operator.sub, [float, float], float)
pset.addPrimitive(operator.mul, [float, float], float)
pset.addPrimitive(protectedDiv, [float, float], float)

# logic operators
# Define a new if-then-else function
def if_then_else(in1, output1, output2):
    if in1: return output1
    else: return output2

pset.addPrimitive(operator.lt, [float, float], bool)
pset.addPrimitive(operator.eq, [float, float], bool)
pset.addPrimitive(if_then_else, [bool, float, float], float)

# terminals
pset.addTerminal(False, bool)
pset.addTerminal(True, bool)
for val in np.arange(-10., 11.):
    pset.addTerminal(val, float)

creator.create('FitnessMax', base.Fitness, weights=(2.0,1.0))
creator.create('Individual', gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('expr', gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
toolbox.register('compile', gp.compile, pset=pset)



row_prob = {} # Each key is row index,
row_mean_prob = {}
def update_true_class_variance(pop_):
    
    """
    pop is a list of indiv.
    Each indiv
    """
    global row_prob, row_mean_prob
    row_prob = {} #reset
    row_mean_prob = {}
    
    for individual in pop_:
        func = toolbox.compile(expr=individual)
        subsample = np.array([func(*record) for record in X_train])

        if X_train[subsample].shape[0] == 0:
            continue

        clf = DecisionTreeClassifier(random_state=34092)
        clf.fit(X_train[subsample], y_train[subsample])

        probas = clf.predict_proba(X_test)
#         print clf.classes_
        for ix, row in enumerate(probas):
#             print row
            try:
                true_p = row[y_test[ix]]
            except:
                true_p = np.nan
            try:
                row_prob[ix].append(true_p)
            except:
                row_prob[ix] = [true_p]
#                 print row_prob[ix]
    
#     print 'row_prob', row_prob
    for key in row_prob.keys():
        row_mean_prob[key] = np.nanmean(row_prob[key])
    
    return    
    

def evaluate_individual(individual):
    global row_mean_prob
    # Transform the tree expression into a callable function
    func = toolbox.compile(expr=individual)
    subsample = np.array([func(*record) for record in X_train])
    
    if X_train[subsample].shape[0] == 0:
        return (1e-20,1e-20)
    
    clf = DecisionTreeClassifier(random_state=34092)
    clf.fit(X_train[subsample], y_train[subsample])
    score = clf.score(X_test, y_test)
    
    probas = clf.predict_proba(X_test)
    total_variance = []
    for ix, row in enumerate(probas):
        try:
            true_p = row[y_test[ix]]
        except:
            true_p = 0
        mean_p = row_mean_prob[ix]
        
        # Can also simply do - 1 - true_p and measure variance in the model.
        if true_p>=0.1:
            added_variance = (mean_p - true_p)**2
        else:
            added_variance = 0 #Should this be nan?
        total_variance.append(added_variance)
    
    mean_variance_added = np.sqrt(np.nanmean(total_variance))
    
    return (score, mean_variance_added)
    
toolbox.register('evaluate', evaluate_individual)
#todo: change this according to multi-objective
# toolbox.register('select', tools.selTournament, tournsize=3)
toolbox.register('select', tools.selNSGA2)

toolbox.register('mate', gp.cxOnePoint)
toolbox.register('expr_mut', gp.genFull, min_=0, max_=3)
toolbox.register('mutate', gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

population = toolbox.population(n=100)
halloffame = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register('std', np.std)
stats.register('min', np.min)
stats.register('avg', np.mean)
stats.register('max', np.max)

clf = DecisionTreeClassifier(random_state=34092)
clf.fit(X_train_tot, y_train_tot)
print('Base DecisionTreeClassifier accuracy: {}'.format(clf.score(X_test_tot, y_test_tot)))

clf = RandomForestClassifier(random_state=34092)
clf.fit(X_train_tot, y_train_tot)
print('Base RandomForestClassifier accuracy: {}'.format(clf.score(X_test_tot, y_test_tot)))

clf = GradientBoostingClassifier(random_state=34092)
clf.fit(X_train_tot, y_train_tot)
print('Base GradientBoostingClassifier accuracy: {}'.format(clf.score(X_test_tot, y_test_tot)))

print('')

cxpb = 0.5
mutpb = 0.5
ngen = 50
verbose = True
holdout_perf = {}

logbook = tools.Logbook()
logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])


update_true_class_variance(population)

# Evaluate the individuals with an invalid fitness ~
invalid_ind = [ind for ind in population if not ind.fitness.valid]
fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
for ind, fit in zip(invalid_ind, fitnesses):
    ind.fitness.values = fit

if halloffame is not None:
    halloffame.update(population)

record = stats.compile(population) if stats else {}
logbook.record(gen=0, nevals=len(invalid_ind), **record)
if verbose:
    print(logbook.stream)

# Begin the generational process
for gen in range(1, ngen + 1):
    # Select the next generation individuals
    offspring = toolbox.select(population, len(population))

    # Vary the pool of individuals
    offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # Update the hall of fame with the generated individuals
    if halloffame is not None:
        halloffame.update(offspring)

    # Replace the current population by the offspring
    population[:] = offspring

    #Generate true class predict_proba variance for each row.
    update_true_class_variance(population)
    
    # Append the current generation statistics to the logbook
    fits = pd.DataFrame()
    fits['score'] = map(lambda x: x.fitness.values[0], population)
    fits['contrib_variance'] = map(lambda x: x.fitness.values[1], population)
    
    record = stats.compile(population) if stats else {}
    logbook.record(gen=gen, nevals=len(invalid_ind),**record)
    holdout_perf[gen] = predict_holdout(population)
    if verbose:
        print(logbook.stream)
        print(fits.score.mean())
        print(fits.contrib_variance.mean())
        print(holdout_perf[gen])
str(halloffame[0])

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Base DecisionTreeClassifier accuracy: 0.851111111111
Base RandomForestClassifier accuracy: 0.955555555556
Base GradientBoostingClassifier accuracy: 0.971111111111

gen	nevals	std     	min  	avg     	max     
0  	100   	0.302632	1e-20	0.270484	0.857567


NameError: name 'pd' is not defined

In [3]:

# pop = offspring[:]
# pop = [halloffame[0]]
halloffame[0].fitness

deap.creator.FitnessMax((0.85756676557863498, 0.31140569213711861))

In [4]:
def predict_holdout(pop):
    forest_predictions = []
    subsample_sizes = []
    for ind_num, individual in enumerate(pop):
        func = toolbox.compile(expr=individual)
    #     print individual
        subsample = np.array([func(*record) for record in X_train])
        subsample_sizes.append(subsample.sum())

        if X_train[subsample].shape[0] == 0:
            continue

        clf = DecisionTreeClassifier(random_state=34092)
        clf.fit(X_train[subsample], y_train[subsample])
        predictions = clf.predict(X_test_tot)
        forest_predictions.append(predictions)
    y_pred = np.array(
    [Counter(instance_forest_predictions).most_common(1)[0][0] for instance_forest_predictions in zip(*forest_predictions)])
    
    return np.sum(y_test_tot == y_pred)*1.0 / len(y_test_tot)

In [5]:
forest_predictions = []
subsample_sizes = []
for ind_num, individual in enumerate(pop):
    func = toolbox.compile(expr=individual)
#     print individual
    subsample = np.array([func(*record) for record in X_train])
    subsample_sizes.append(subsample.sum())
    
    if X_train[subsample].shape[0] == 0:
        continue
    
    clf = DecisionTreeClassifier(random_state=34092, max_depth=5)
    clf.fit(X_train[subsample], y_train[subsample])
    predictions = clf.predict(X_test_tot)
    forest_predictions.append(predictions)

NameError: name 'pop' is not defined

In [468]:
# for x in 
fits = pd.DataFrame()
fits['score'] = map(lambda x: x.fitness.values[0], pop)
fits['contrib_variance'] = map(lambda x: x.fitness.values[1], pop)

In [469]:
%matplotlib inline
import matplotlib.pyplot as plt
# fits.contrib_variance.hist()

In [470]:
fits.score.mean()


0.78528189910979218

In [471]:

# plt.hist(subsample_sizes)
print np.mean(subsample_sizes)
print np.std(subsample_sizes)

863.09
100.248001975


In [472]:
import pandas as pd
## Mean # of unique labels predicted per sample.
pd.DataFrame(forest_predictions).apply(lambda x: len(x.unique()), axis=0).mean()

2.3569844789356984

In [473]:
from collections import Counter
from sklearn.metrics import accuracy_score

y_pred = np.array(
    [Counter(instance_forest_predictions).most_common(1)[0][0] for instance_forest_predictions in zip(*forest_predictions)])
np.sum(y_test_tot == y_pred)*1.0 / len(y_test_tot)

0.80487804878048785

In [465]:
pd.DataFrame(forest_predictions)

,0,1,2,3,4,5,6,7,8,9,...,441,442,443,444,445,446,447,448,449,450
0,6,6,1,0,1,8,2,2,7,4,...,8,8,0,5,2,2,9,9,3,4


In [475]:
pd.DataFrame(forest_predictions)

,0,1,2,3,4,5,6,7,8,9,...,441,442,443,444,445,446,447,448,449,450
0,6,6,1,0,1,8,2,2,7,4,...,8,8,0,5,2,2,9,9,3,4
1,6,6,1,0,1,8,2,2,7,4,...,8,8,0,5,2,2,9,9,3,4
2,6,6,1,0,1,8,2,2,7,4,...,8,8,0,5,2,2,9,9,3,4
3,6,6,1,0,1,8,2,2,7,4,...,8,8,8,5,2,2,9,9,3,4
4,6,6,1,0,1,8,2,2,7,4,...,8,8,0,5,2,2,9,9,3,4
5,6,6,1,0,1,8,2,2,7,4,...,8,8,0,5,2,2,9,9,3,1
6,6,6,1,0,1,8,2,2,7,4,...,8,8,0,5,2,2,9,9,3,4
7,6,6,1,0,1,8,2,2,7,4,...,8,8,0,5,2,2,9,9,3,4
8,6,6,1,0,1,8,2,2,7,4,...,8,8,0,5,2,2,9,9,3,4
9,6,6,1,0,1,8,2,2,7,4,...,8,8,0,5,2,2,9,9,3,1


In [97]:
from sklearn import __version__

In [98]:
__version__

'0.17.1'